In [1]:
import sys
import os
import pandas as pd
import json

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

# Now you can import the IntentClassifier from src/infer_intent.py
from src.infer_intent import IntentClassifier
from tqdm import tqdm
import pandas as pd

In [2]:
# with open('../data/internal/search_examples.txt', 'r', encoding='utf-8') as f:
#     yelp_texts = f.read().split('\n')

In [3]:
# yelp_texts[:10]

## Using yelp Keywords for validation

In [4]:
tmp2 = pd.read_json("https://firefox-settings-attachments.cdn.mozilla.net/main-workspace/quicksuggest/33987d71-9e87-4b7e-86d3-6f292b89e8bf.json")
yelp_texts = tmp2['subjects'][0]

In [5]:
len(tmp2['subjects'][0])

2870

In [6]:
# !ls -ltrsh models/

In [7]:
cls = IntentClassifier()

ONNX model downloaded.


#### Yelp internal data evaluation (python)

In [8]:
yelp_hit_count = 0
yelp_queries_misclassified = []
yelp_queries_pred_res = []
for query in tqdm(yelp_texts):
    pred_result, pred_proba = cls.find_intent(query)
    if pred_result == 'yelp_intent':
        yelp_hit_count += 1
    else:
        yelp_queries_misclassified.append({'query': query, 'pred_result': pred_result, 'pred_proba': pred_proba[pred_result]})
    yelp_queries_pred_res.append({'query': query, 'pred_result': pred_result, 'pred_proba': pred_proba[pred_result]})
print(f"yelp hit rate (accuracy) = {yelp_hit_count/len(yelp_texts)}")

  2%|██▉                                                                                                                                                                      | 49/2870 [00:00<00:42, 65.76it/s]/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: overflow encountered in exp
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: invalid value encountered in divide
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2870/2870 [00:43<00:00, 65.27it/s]

yelp hit rate (accuracy) = 0.9355400696864111


In [9]:
yelp_queries_pred_res_df = pd.DataFrame(yelp_queries_pred_res)
yelp_queries_pred_res_df
# .groupby('pred_result')['pred_proba'].describe()

,query,pred_result,pred_proba
0,24 hour cleaning services,yelp_intent,1.000
1,24 hour maid service,yelp_intent,0.975
2,24 hour pharmacy,yelp_intent,0.999
3,24 hour restaurants,yelp_intent,1.000
4,24 hour walmart,navigation_intent,0.761
...,...,...,...
2865,yoga,yelp_intent,0.901
2866,yogurstory,yelp_intent,0.786
2867,you move me,yelp_intent,0.614
2868,yume wo katare,yelp_intent,0.676


In [10]:
yelp_queries_pred_res_df.groupby('pred_result')['pred_proba'].describe(percentiles=[.05, .1,.2,.25,.3, .4,.5,.6, .7, .75, .8, .9, .95, .98, .99])

,count,mean,std,min,5%,10%,20%,25%,30%,40%,50%,60%,70%,75%,80%,90%,95%,98%,99%,max
pred_result,,,,,,,,,,,,,,,,,,,,
information_intent,75.0,0.734067,0.165696,0.345,0.42590,0.4862,0.6062,0.62400,0.6584,0.6996,0.7550,0.8048,0.8372,0.86750,0.8936,0.9296,0.96260,0.98472,0.99026,0.991
navigation_intent,8.0,0.684125,0.201880,0.390,0.43585,0.4817,0.5346,0.54650,0.5577,0.5766,0.6715,0.7706,0.8042,0.82600,0.8498,0.9073,0.94265,0.96386,0.97093,0.978
purchase_intent,10.0,0.531500,0.189461,0.265,0.29245,0.3199,0.3892,0.40700,0.4106,0.4484,0.4960,0.5572,0.6499,0.70525,0.7396,0.7597,0.78535,0.80074,0.80587,0.811
translation_intent,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
travel_intent,21.0,0.688714,0.190144,0.398,0.41100,0.4580,0.5280,0.53700,0.5500,0.6340,0.6680,0.6820,0.8040,0.86500,0.9120,0.9550,0.96800,0.97520,0.97760,0.980
unknown,22.0,0.606636,0.214391,0.228,0.32735,0.3547,0.4006,0.48225,0.4923,0.5140,0.5755,0.6664,0.7280,0.76175,0.8340,0.9240,0.93000,0.93696,0.93948,0.942
weather_intent,4.0,0.626000,0.220675,0.397,0.40900,0.4210,0.4450,0.45700,0.4690,0.5442,0.6450,0.7458,0.8134,0.81400,0.8146,0.8158,0.81640,0.81676,0.81688,0.817
yelp_intent,2683.0,0.963171,0.095206,0.339,0.73610,0.8890,0.9734,0.98600,0.9920,0.9970,0.9990,0.9990,1.0000,1.00000,1.0000,1.0000,1.00000,1.00000,1.00000,1.000


In [11]:
yelp_queries_pred_res_df.loc[yelp_queries_pred_res_df['pred_proba'] < 0.8].sort_values('pred_proba', ascending=False)[:60]

,query,pred_result,pred_proba
955,han il kwan,information_intent,0.799
1084,here fishy fishy,yelp_intent,0.799
10,absolute bagels,information_intent,0.796
1658,mini golf,yelp_intent,0.795
2081,petpal la,yelp_intent,0.794
1354,indoor mini golf,yelp_intent,0.793
823,flowers,yelp_intent,0.792
1592,maid service rates,yelp_intent,0.791
1440,interstate moving,yelp_intent,0.788
1617,medical supply store,yelp_intent,0.787


In [12]:
## old
# yelp hit rate (accuracy) = 0.3256021409455843

## new (Mozilla/mobilebert-uncased-finetuned-LoRA-intent-classifier)
# yelp hit rate (accuracy) = 0.6703835860838537
# yelp hit rate (accuracy) = 0.7832292595896521
# yelp hit rate (accuracy) = 0.8862622658340767
# yelp hit rate (accuracy) = 0.8742160278745644
# yelp hit rate (accuracy) = 0.9243902439024391

In [13]:
yelp_queries_misclassified_df = pd.DataFrame(yelp_queries_misclassified)
print(f"Number of yelp queries misclassified = {len(yelp_queries_misclassified_df)} out of {len(yelp_texts)} examples")
yelp_queries_misclassified_df

Number of yelp queries misclassified = 185 out of 2870 examples


,query,pred_result,pred_proba
0,24 hour walmart,navigation_intent,0.761
1,absolute bagels,information_intent,0.796
2,all that shabu,information_intent,0.893
3,aloha mamacita,information_intent,0.840
4,american cruise lines,travel_intent,0.968
...,...,...,...
180,where to find a painter,information_intent,0.532
181,where to hire a painter,navigation_intent,NaN
182,wicked spoon,unknown,0.493
183,wings,information_intent,0.669


#### Load the Fx ML inference output (using Fx js)

In [14]:
fx_ml_intent = pd.read_json("../data/ML_output_YELP_KEYWORDS_DATA.json")
fx_ml_hit_rate_for_yelp_intent = float(fx_ml_intent['intent'].value_counts(normalize=True)['yelp_intent'])
print(f"Fx ML yelp intent hit rate = {fx_ml_hit_rate_for_yelp_intent}")

Fx ML yelp intent hit rate = 0.926829268292683


## Using Yelp additional validation dataset

In [15]:
tqdm.pandas()

In [16]:
yelp_val_generated_data = pd.read_csv("../data/yelp_val_generated_data.csv")
yelp_val_generated_data['pred_result'] = yelp_val_generated_data['queries'].progress_apply(lambda query: cls.find_intent(query)[0])
hit_rate = float(yelp_val_generated_data['pred_result'].value_counts(normalize=True)['yelp_intent'])
print(f"yelp generated val data hit rate (accuracy) = {hit_rate}")

  2%|██▌                                                                                                                                                                      | 30/2000 [00:00<00:29, 66.53it/s]/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: overflow encountered in exp
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: invalid value encountered in divide
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:30<00:00, 65.02it/s]

yelp generated val data hit rate (accuracy) = 0.956


In [17]:
# yelp generated val data hit rate (accuracy) = 0.982

In [18]:
queries_array = yelp_val_generated_data['queries'].values

# Create a dictionary structure
data = {
    "queries": queries_array.tolist()  
}

# Define the output JSON file path
output_file_path = "../data/yelp_val_generated_data.json"

## Uncomment to write it else its almost static dataset
# # Write to a JSON file
# with open(output_file_path, "w") as json_file:
#     json.dump(data, json_file, indent=2)


In [19]:
## yelp validation from Fx MLSuggest

fx_ml_intent_val = pd.read_json("../data/ML_output_YELP_VAL_DATA.json")
fx_ml_hit_rate_for_yelp_intent_2 = float(fx_ml_intent_val['intent'].value_counts(normalize=True)['yelp_intent'])
print(f"Fx ML yelp intent2 hit rate = {fx_ml_hit_rate_for_yelp_intent_2}")

Fx ML yelp intent2 hit rate = 0.959


#### Weather internal data evaluation

In [20]:
with open('../data/internal/weather_search_examples.txt', 'r', encoding='utf-8') as f:
    weather_texts = f.read().split('\n')

In [21]:
weather_hit_count = 0
weather_queries_misclassified = []
weather_queries_pred_res = []
for query in tqdm(weather_texts):
    pred_result, pred_proba = cls.find_intent(query)
    if pred_result == 'weather_intent':
        weather_hit_count += 1
    else:
        weather_queries_misclassified.append({'query': query, 'pred_result': pred_result, 'pred_proba': pred_proba[pred_result]})
    weather_queries_pred_res.append({'query': query, 'pred_result': pred_result, 'pred_proba': pred_proba[pred_result]})
print(f"weather hit rate (accuracy) = {weather_hit_count/len(weather_texts)}")

 17%|█████████████████████████████▎                                                                                                                                              | 7/41 [00:00<00:00, 65.56it/s]/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: overflow encountered in exp
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: invalid value encountered in divide
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:00<00:00, 65.80it/s]

weather hit rate (accuracy) = 0.7317073170731707


In [22]:
## old
# weather hit rate (accuracy) = 0.3902439024390244

## new (Mozilla/mobilebert-uncased-finetuned-LoRA-intent-classifier)
# weather hit rate (accuracy) = 0.4878048780487805
# weather hit rate (accuracy) = 0.6341463414634146
# weather hit rate (accuracy) = 0.6585365853658537
# weather hit rate (accuracy) = 0.8048780487804879


In [23]:
weather_queries_misclassified_df = pd.DataFrame(weather_queries_misclassified)
print(f"Number of weather queries misclassified = {len(weather_queries_misclassified)} out of {len(weather_texts)} examples")
weather_queries_misclassified_df

Number of weather queries misclassified = 11 out of 41 examples


,query,pred_result,pred_proba
0,forcast,translation_intent,NaN
1,wather,unknown,0.280
2,vindy,yelp_intent,0.795
3,sunny,translation_intent,NaN
4,air quality,yelp_intent,0.730
5,pollen,information_intent,0.736
6,tiempo,yelp_intent,0.959
7,clima,unknown,0.367
8,doppler radar,information_intent,0.711
9,map,information_intent,0.775


In [24]:
pd.DataFrame(weather_queries_pred_res)['pred_proba'].describe(percentiles=[.1,.2,.25,.3, .4,.5,.6, .7, .75, .8, .9, .95, .98, .99])

count    39.000000
mean      0.872385
std       0.171236
min       0.280000
10%       0.681000
20%       0.759400
25%       0.804500
30%       0.837400
40%       0.874200
50%       0.959000
60%       0.971800
70%       0.995800
75%       0.997000
80%       0.997400
90%       0.999200
95%       1.000000
98%       1.000000
99%       1.000000
max       1.000000
Name: pred_proba, dtype: float64

#### Weather validate the generated data with pre and post modifiers

In [25]:
weather_val_generated_data = pd.read_csv("../data/weather_val_generated_data.csv")
weather_val_generated_data['pred_result'] = weather_val_generated_data['queries'].progress_apply(lambda query: cls.find_intent(query)[0])
hit_rate = float(weather_val_generated_data['pred_result'].value_counts(normalize=True)['weather_intent'])
print(f"weather generated val data hit rate (accuracy) = {hit_rate}")

  7%|███████████▎                                                                                                                                                            | 135/2000 [00:02<00:28, 65.90it/s]/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: overflow encountered in exp
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: invalid value encountered in divide
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:30<00:00, 64.94it/s]

weather generated val data hit rate (accuracy) = 0.7205


In [26]:
# weather generated val data hit rate (accuracy) = 0.643
# weather generated val data hit rate (accuracy) = 0.79

#### Orcas golden dataset for intent validation

In [27]:
orcas_data = pd.read_csv("../data/ORCAS_golden_dataset.tsv", sep='\t')
print(len(orcas_data))

1000


In [28]:
orcas_data.head()

,query,url,label_manual,target
0,best reads,http://thegreatestbooks.org/,Abstain,information_intent
1,tamerind,https://en.wikipedia.org/wiki/Tamarind,Factual,information_intent
2,show mi ip,http://showip.net/,Instrumental,navigation_intent
3,do carpenter ants eat wood,https://doyourownpestcontrol.com/carp.htm,Factual,information_intent
4,rheumatoid arthritis in children,https://www.webmd.com/rheumatoid-arthritis/und...,Abstain,information_intent


In [29]:
orcas_data['target'].value_counts()

target
information_intent    591
navigation_intent     265
unknown                78
purchase_intent        27
yelp_intent            15
weather_intent         14
travel_intent           9
translation_intent      1
Name: count, dtype: int64

In [30]:
# orcas_data['query'].values

intent_hit_count = 0
intent_queries_misclassified = []
intent_queries_classified = []
intent_queries_below_thresholds = []
skipped_for_threshold = 0
for query, target in tqdm(orcas_data[['query', 'target']].values.tolist()):
    pred_result, pred_proba = cls.find_intent(query)
    ## TODO
    if pred_proba[pred_result] > 0.5:
        intent_queries_classified.append({'query': query, 'pred_result': pred_result, 'target': target, 'pred_proba': pred_proba[pred_result]})
        if pred_result == target:
            intent_hit_count += 1
        else:
            intent_queries_misclassified.append({'query': query, 'pred_result': pred_result, 'target': target})
    else:
        skipped_for_threshold += 1
        intent_queries_below_thresholds.append({'query': query, 'pred_result': pred_result, 'target': target, 'pred_proba': pred_proba[pred_result]})
print(f"intent hit rate (accuracy) = {intent_hit_count/len(orcas_data)}")

  0%|                                                                                                                                                                                  | 0/1000 [00:00<?, ?it/s]/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: overflow encountered in exp
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: invalid value encountered in divide
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:15<00:00, 65.15it/s]

intent hit rate (accuracy) = 0.517


In [31]:
# intent hit rate (accuracy) = 0.521

In [32]:
skipped_for_threshold

188

In [33]:
pd.DataFrame(intent_queries_below_thresholds)

,query,pred_result,target,pred_proba
0,tamerind,travel_intent,information_intent,NaN
1,rheumatoid arthritis in children,translation_intent,information_intent,NaN
2,outlamder,unknown,unknown,0.346
3,is richard gere dead,unknown,information_intent,NaN
4,cdczika,unknown,navigation_intent,0.441
...,...,...,...,...
183,usaa san antonio,information_intent,navigation_intent,0.338
184,news sport football,translation_intent,information_intent,NaN
185,alveoli structure and function,navigation_intent,information_intent,NaN
186,first health insurance reviews,information_intent,information_intent,0.325


In [34]:
pd.DataFrame(intent_queries_below_thresholds).groupby('pred_result')['pred_proba'].describe(percentiles=[.05, .1,.2,.25,.3, .4,.5,.6, .7, .75, .8, .9, .95, .98, .99])

,count,mean,std,min,5%,10%,20%,25%,30%,40%,50%,60%,70%,75%,80%,90%,95%,98%,99%,max
pred_result,,,,,,,,,,,,,,,,,,,,
information_intent,38.0,0.425658,0.056925,0.325,0.33665,0.3557,0.3614,0.37175,0.3874,0.4046,0.4285,0.4682,0.4719,0.47825,0.4800,0.4906,0.49375,0.49852,0.49926,0.500
navigation_intent,9.0,0.404444,0.046763,0.325,0.33900,0.3530,0.3648,0.36800,0.3768,0.3934,0.4070,0.4342,0.4428,0.44400,0.4440,0.4474,0.45420,0.45828,0.45964,0.461
purchase_intent,10.0,0.389600,0.075851,0.269,0.27440,0.2798,0.3210,0.34125,0.3597,0.3810,0.4050,0.4242,0.4344,0.44700,0.4584,0.4761,0.47655,0.47682,0.47691,0.477
translation_intent,3.0,0.405333,0.087848,0.304,0.31880,0.3336,0.3632,0.37800,0.3928,0.4224,0.4520,0.4536,0.4552,0.45600,0.4568,0.4584,0.45920,0.45968,0.45984,0.460
travel_intent,10.0,0.398400,0.058525,0.302,0.30830,0.3146,0.3576,0.37075,0.3757,0.3976,0.4140,0.4200,0.4236,0.42450,0.4300,0.4543,0.47365,0.48526,0.48913,0.493
unknown,9.0,0.405444,0.057090,0.334,0.33480,0.3356,0.3420,0.34600,0.3640,0.3946,0.4090,0.4346,0.4422,0.44300,0.4518,0.4688,0.47640,0.48096,0.48248,0.484
weather_intent,3.0,0.456000,0.017059,0.442,0.44290,0.4438,0.4456,0.44650,0.4474,0.4492,0.4510,0.4558,0.4606,0.46300,0.4654,0.4702,0.47260,0.47404,0.47452,0.475
yelp_intent,25.0,0.433320,0.043974,0.340,0.36480,0.3788,0.3994,0.40600,0.4084,0.4216,0.4360,0.4490,0.4646,0.47100,0.4722,0.4894,0.49460,0.49760,0.49880,0.500


In [35]:
intent_queries_classified_df = pd.DataFrame(intent_queries_classified)
intent_queries_misclassified_df = pd.DataFrame(intent_queries_misclassified)
# intent_queries_misclassified_df

In [36]:
intent_queries_misclassified_df['target'].value_counts()

target
information_intent    122
navigation_intent      99
unknown                57
purchase_intent        10
yelp_intent             3
travel_intent           3
translation_intent      1
Name: count, dtype: int64

In [37]:
pd.crosstab(intent_queries_classified_df['target'], intent_queries_classified_df['pred_result'])

pred_result,information_intent,navigation_intent,purchase_intent,translation_intent,travel_intent,unknown,weather_intent,yelp_intent
target,,,,,,,,
information_intent,354,32,8,2,8,8,3,61
navigation_intent,44,117,4,0,6,4,1,40
purchase_intent,5,1,14,0,0,0,0,4
translation_intent,1,0,0,0,0,0,0,0
travel_intent,1,0,0,0,4,0,0,2
unknown,33,5,3,0,1,2,0,15
weather_intent,0,0,0,0,0,0,14,0
yelp_intent,3,0,0,0,0,0,0,12


In [38]:
intent_queries_misclassified_df.loc[intent_queries_misclassified_df['target'] == 'yelp_intent']

,query,pred_result,target
9,almond butter,information_intent,yelp_intent
172,health insurance plans,information_intent,yelp_intent
183,gordo's,information_intent,yelp_intent


In [39]:
intent_queries_misclassified_df.loc[intent_queries_misclassified_df['target'] == 'navigation_intent']

,query,pred_result,target
1,show mi ip,yelp_intent,navigation_intent
8,apple phone replacement,yelp_intent,navigation_intent
11,paycvor,unknown,navigation_intent
13,background sound effects free download,information_intent,navigation_intent
15,dell support assistant repair,yelp_intent,navigation_intent
...,...,...,...
286,windows 10 turn off screen lock,information_intent,navigation_intent
288,starwars disneyland,travel_intent,navigation_intent
289,ncdps forms,information_intent,navigation_intent
292,riverside county ca property tax search,information_intent,navigation_intent


In [40]:
intent_queries_misclassified_df.loc[intent_queries_misclassified_df['target'] == 'unknown']

,query,pred_result,target
44,chilie,yelp_intent,unknown
45,mr hans,information_intent,unknown
47,what is this ip,information_intent,unknown
49,rmb and cny,navigation_intent,unknown
65,dcm,yelp_intent,unknown
74,i make mistakes,information_intent,unknown
76,pho delivery,yelp_intent,unknown
77,who sang it,information_intent,unknown
83,j,yelp_intent,unknown
87,no till food plot seed,information_intent,unknown


In [41]:
from sklearn.metrics import classification_report

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [42]:
print(classification_report(intent_queries_classified_df['target'],
                            intent_queries_classified_df['pred_result']))

                    precision    recall  f1-score   support

information_intent       0.80      0.74      0.77       476
 navigation_intent       0.75      0.54      0.63       216
   purchase_intent       0.48      0.58      0.53        24
translation_intent       0.00      0.00      0.00         1
     travel_intent       0.21      0.57      0.31         7
           unknown       0.14      0.03      0.05        59
    weather_intent       0.78      1.00      0.88        14
       yelp_intent       0.09      0.80      0.16        15

          accuracy                           0.64       812
         macro avg       0.41      0.53      0.42       812
      weighted avg       0.71      0.64      0.66       812



In [43]:
# print(classification_report(intent_queries_classified_df['target'],
#                             intent_queries_classified_df['pred_result']))

### with no thresholds

#                     precision    recall  f1-score   support

# information_intent       0.80      0.70      0.75       591
#  navigation_intent       0.69      0.55      0.61       265
#    purchase_intent       0.45      0.52      0.48        27
# translation_intent       0.50      1.00      0.67         1
#      travel_intent       0.09      0.56      0.16         9
#            unknown       0.21      0.08      0.11        78
#     weather_intent       0.57      0.93      0.70        14
#        yelp_intent       0.09      0.80      0.16        15

#           accuracy                           0.61      1000
#          macro avg       0.43      0.64      0.46      1000
#       weighted avg       0.70      0.61      0.64      1000

In [44]:
intent_queries_misclassified_df.loc[intent_queries_misclassified_df['pred_result'] == 'yelp_intent']['target'].value_counts()

target
information_intent    61
navigation_intent     40
unknown               15
purchase_intent        4
travel_intent          2
Name: count, dtype: int64

In [45]:
intent_queries_classified_df.groupby('pred_result')['pred_proba'].describe(percentiles=[.05, .1,.2,.25,.3, .4,.5,.6, .7, .75, .8, .9, .95, .98, .99])

,count,mean,std,min,5%,10%,20%,25%,30%,40%,50%,60%,70%,75%,80%,90%,95%,98%,99%,max
pred_result,,,,,,,,,,,,,,,,,,,,
information_intent,441.0,0.891952,0.138047,0.501,0.58600,0.6580,0.7810,0.83000,0.8690,0.9250,0.9580,0.9800,0.9930,0.99600,0.9980,1.0000,1.00000,1.00000,1.00000,1.000
navigation_intent,155.0,0.894232,0.135502,0.512,0.60700,0.6750,0.7734,0.84050,0.8746,0.9362,0.9640,0.9798,0.9930,0.99600,0.9980,0.9996,1.00000,1.00000,1.00000,1.000
purchase_intent,29.0,0.734448,0.154549,0.518,0.52020,0.5284,0.5742,0.59400,0.6300,0.6576,0.7760,0.8042,0.8176,0.82600,0.8888,0.9350,0.96920,0.98084,0.98392,0.987
translation_intent,2.0,0.698500,0.036062,0.673,0.67555,0.6781,0.6832,0.68575,0.6883,0.6934,0.6985,0.7036,0.7087,0.71125,0.7138,0.7189,0.72145,0.72298,0.72349,0.724
travel_intent,19.0,0.794053,0.160364,0.552,0.57180,0.5764,0.6404,0.66750,0.6746,0.7324,0.8310,0.8594,0.9232,0.94100,0.9596,0.9934,0.99900,0.99900,0.99900,0.999
unknown,14.0,0.753143,0.139711,0.552,0.58125,0.6033,0.6276,0.63975,0.6547,0.7106,0.7300,0.7494,0.8313,0.86575,0.8966,0.9419,0.96330,0.97812,0.98306,0.988
weather_intent,18.0,0.905556,0.134964,0.637,0.66675,0.6839,0.7648,0.79975,0.8771,0.9754,0.9990,0.9992,1.0000,1.00000,1.0000,1.0000,1.00000,1.00000,1.00000,1.000
yelp_intent,134.0,0.781007,0.163846,0.508,0.52265,0.5486,0.5934,0.61400,0.6620,0.7564,0.8095,0.8650,0.9053,0.93900,0.9536,0.9827,0.99400,0.99734,0.99934,1.000


In [46]:
# intent_queries_misclassified_df.loc[(intent_queries_misclassified_df['target'] == 'information_intent') &
#                                     (intent_queries_misclassified_df['pred_result'] == 'yelp_intent')]['query'].values